## Load modules

In [212]:
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone, utc
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import statsmodels.formula.api as smf
from copy import deepcopy
import re
from itertools import compress
from scipy import stats

import Scripts.config as config
import Scripts.Functions as func
from app import getConfigData
from Scripts.ProcessData_resampler import setUTCDatetime

## Functions and setup

In [213]:
list(config.config['info']['colours'].columns)

['colour', 'theme', 'r', 'g', 'b', 'rgb', 'rgba_str']

In [214]:
def makeResDict(ChivasB):
    if ChivasB == True:
        resistor_list = [13.5,28.5,54.5,104,153.5,225,303.6,406.1,514.5,755,1004,2004,5100,15000,51000]
        resistor_noms2 = ['10','24.9','51','100','150','221','300','402','510','750','1000','2000','5100','15000','51000']
    else:
        resistor_list = [10.5,51.4,150.1,300,508.2,998,1996,5088,14916,51101]
        resistor_noms2 = ['10','51','150','300','510','1000','2000','5100','15000','51000']

    return dict(zip(resistor_noms2, resistor_list))

def getColour(theme, colour):
    rgba_str = config.config['info']['colours'].query("theme == @theme").query("colour == @colour")['rgba_str'].values[0]
    return rgba_str

def dropIncompleteRows(data, cols_list):
    data.drop(data[data[cols_list].isnull().any(axis=1)].index, inplace=True)
    data.reset_index(inplace=True, drop=True)
    return data

def subsetDataCols(data, cols_list):
    data = deepcopy(data[cols_list])
    data = dropIncompleteRows(data, cols_list)
    return data

def subsetDataBetweenDates(data, start, end):
    return deepcopy(data.loc[data.DateTime >= start].loc[data.DateTime < end])

def getOCPOnIndices(data, cell):
    #Find OCP Ons
    return list(data[(data[ocp_ds[cell]] > 0)].index)

def getOCPOffIndices(data, cell):
    #Find OCP Offs
    return list(data[(data[ocp_ds[cell]] < 0)].index)

def getOCPOnOffDict(data, cell):
    OCP_ons = getOCPOnIndices(data, cell)
    ocp_onoff = {}
    #Find OCP Offs
    OCP_offs = getOCPOffIndices(data, cell)
    #For each OCP On
    for on_id in range(0,len(OCP_ons)):
        OCP_on = np.int64(OCP_ons[on_id])
        #Set next OCP On
        if on_id == len(OCP_ons)-1:
            next_OCP_on = np.int64(data.index[len(data)-1])
        else:
            next_OCP_on =  np.int64(OCP_ons[on_id + 1])
            #Find OCP Offs after OCP On
        ocp_offs_after_on = list(compress(OCP_offs, OCP_offs > OCP_on))
        if len(ocp_offs_after_on) > 0:
            #Find OCP Offs before next OCP On
            ocp_offs_before_next = list(compress(ocp_offs_after_on, ocp_offs_after_on < next_OCP_on))
            if len(ocp_offs_before_next) > 0:
                #Save to dict
                ocp_onoff[OCP_on] = ocp_offs_before_next[0]
    return ocp_onoff

def getOCPResIndices(data, cell):
    ocp_onoff = getOCPOnOffDict(data, cell)
    ocp_on_dict = {}
    ocp_off_dict = {}
    res_inc_dict = {}
    #Find resistance decrements and increments
    res_decs = list(data[(data[rext_ds[cell]] < 0)].index)
    res_incs = list(data[(data[rext_ds[cell]] > 0)].index)

    n = 0
    #For each OCP on
    for on_id in range(0,len(ocp_onoff.keys())):
        OCP_on = np.int64(list(ocp_onoff.keys())[on_id])
        OCP_off = np.int64(ocp_onoff[OCP_on])
        #Set next OCP on
        if on_id == len(ocp_onoff.keys())-1:
            next_OCP_on = np.int64(data.index[len(data)-1])
        else:
            next_OCP_on =  np.int64(list(ocp_onoff.keys())[on_id + 1])
        #Find resistance decrements after OCP Off
        res_dec_after_off = list(compress(res_decs, res_decs > OCP_off))
        if len(res_dec_after_off) > 0:
            #Find resistance decrements before next OCP On
            res_dec_before_next_ocp = list(compress(res_dec_after_off, res_dec_after_off < next_OCP_on))
            if len(res_dec_before_next_ocp) > 0:
                #Find resistance increments after resistance decrement
                res_dec = np.int64(res_dec_before_next_ocp[0])
                res_inc_after_dec = list(compress(res_incs, res_incs > res_dec))
                if len(res_inc_after_dec) > 0:
                    #Find resistance increments before next OCP On
                    res_inc_before_next_ocp = list(compress(res_inc_after_dec, res_inc_after_dec < next_OCP_on))
                    if len(res_inc_before_next_ocp) > 0:
                        #Save OCP + Res indices to dicts
                        ocp_on_dict[n] = OCP_on
                        ocp_off_dict[n] = OCP_off
                        res_inc_dict[n] = res_inc_before_next_ocp[0]
                        n += 1

    return pd.DataFrame.from_dict({"OCP_ON":ocp_on_dict, "OCP_OFF":ocp_off_dict, "RES_INC":res_inc_dict}, orient='columns')

def getTimings(data):
    timings = {}
    for cell in cells:
        timings_index = getOCPResIndices(data, cell)
        col_dict = {}
        for col in timings_index:
            col_dict[col] = data.loc[timings_index[col], 'DateTime'].values
            col_dict[col + "_index"] = timings_index[col]
        col_dict['DURATION'] = col_dict["RES_INC"] - col_dict["OCP_ON"]
        timings[cell] = pd.DataFrame.from_dict(col_dict, orient='columns')
    return timings

def alignPCIndices(timings):
    aligned_indices = {}

    for cell in cells:
        aligned_indices[cell] = {}
        for pc_id in timings[cells[0]]['OCP_ON'].index:
            if cell == cells[0]:
                aligned_indices[cell][pc_id] = pc_id
            else:
                start = timings[cells[0]]['OCP_ON'][pc_id] - same_pc_threshold
                end = timings[cells[0]]['OCP_ON'][pc_id] + same_pc_threshold
                try:
                    aligned_indices[cell][pc_id] = timings[cell].query("OCP_ON > @start").query("OCP_ON < @end").index[0]
                except:
                    pass
    aligned_indices_df =  pd.DataFrame.from_dict(aligned_indices, orient='columns')
    dropIncompleteRows(aligned_indices_df, aligned_indices_df.columns)
    aligned_indices_df = aligned_indices_df.astype(np.int64)
    return aligned_indices_df

def alignTimings(timings):
    aligned_indices_df = alignPCIndices(timings)
    aligned_timings = {}
    for cell in cells:
        aligned_timings[cell] = timings[cell][timings[cell].index.isin(aligned_indices_df[cell].values)].reset_index(drop=True)
    return aligned_timings

def getPCData(data, cols_list, start, end):
    pc_data = subsetDataBetweenDates(data, start, end)
    pc_data = subsetDataCols(pc_data, cols_list)
    return pc_data

def getPCDataDict(data, cell, timings):
    pc_data_dict = {}
    for pc_id, ind in timings.iterrows():
        start = data.loc[ind.OCP_ON_index, 'DateTime']
        end = data.loc[ind.RES_INC_index, 'DateTime']
        cols_list = ['DateTime', vols[cell], ocps[cell], rexts[cell]]
        pc_data_dict[pc_id] = getPCData(data, cols_list, start, end)
    return pc_data_dict

def getRextVolData(pc_data, cell, rext):
    if rext == 'OCP':
        sub_data = list(pc_data.loc[pc_data[ocps[cell]] == 1][vols[cell]])
    else:
        sub_data = list(pc_data.loc[pc_data[rexts[cell]] == rext][vols[cell]])
    return sub_data

def getLastValue(pc_data, cell, rext):
    sub_data = getRextVolData(pc_data, cell, rext)
    return sub_data[len(sub_data)-1]

def getRextStr(rext):
    return str(rext).rstrip('0').rstrip('.') if '.' in str(rext) else str(rext)

def getUniqueRexts(pc_data, cell):
    return pc_data.loc[pc_data[ocps[cell]] == 0][rexts[cell]].unique()

def CalcCurrentuA(vol, rext_str):
    #vol in mV
    #rext in ohms
    return vol / res_dict[rext_str] * 1000 # in uA

def CalcPoweruW(vol, rext_str="", cur=""):
    #vol in mV
    #rext in ohms
    #cur in uA
    if rext_str == "":
        return (vol * cur) / 1000 # in uW
    else:
        return (vol * vol / res_dict[rext_str])
    
def getPCSummary(pc_data, cell):
    #Summary dicts
    pc_vol = {} #mV
    pc_cur = {} #uA
    pc_pow = {} #uW

    #Get unique rext vals
    rext_vals = getUniqueRexts(pc_data, cell)

    #Set OCP vals
    pc_vol['OCP'] = getLastValue(pc_data, cell, 'OCP')
    pc_cur['OCP'] = 0
    pc_pow['OCP'] = 0

    prev_vol = pc_vol['OCP']

    for rext in rext_vals:
        vol = getLastValue(pc_data, cell, rext)
        vol_data = getRextVolData(pc_data, cell, rext)
        if vol < prev_vol: #Only decrements
            if len(vol_data) > 1: # Fix for failed resistor changes
                rext_str = getRextStr(rext)
                pc_vol[rext_str] = vol
                pc_cur[rext_str] = CalcCurrentuA(vol = pc_vol[rext_str], rext_str = rext_str) # in uA
                pc_pow[rext_str] = CalcPoweruW(vol = pc_vol[rext_str], rext_str = rext_str) # in uW

    if len(pc_vol) > 1: #More than just OCP
        return pd.DataFrame.from_dict({"Voltage":pc_vol, "Current":pc_cur, "Power":pc_pow}, orient='columns')

def getIndivSummary():
    global aligned_timings
    pc_data_dict = {}
    pc_summary = {}
    pc_ids_todrop = []
    for cell in cells:
        pc_data_dict[cell] = getPCDataDict(data, cell, aligned_timings[cell])
        for pc_id in pc_data_dict[cell]:
            if cell == cells[0]:
                pc_summary[pc_id] = {}
            pc_summary[pc_id][cell] = getPCSummary(pc_data_dict[cell][pc_id], cell)
            if pc_summary[pc_id][cell] is None:
                if pc_id not in pc_ids_todrop:
                    pc_ids_todrop.append(pc_id)

    for pc_id in pc_ids_todrop:
        print("Dropping PC_ID = " + str(pc_id))
        for cell in cells:
            pc_summary.pop(pc_id, None)
            pc_data_dict[cell].pop(pc_id, None)
            aligned_timings[cell].drop(pc_id, inplace=True)
    return pc_summary

def createPCSums():
    pc_ind_summary = getIndivSummary()
    pc_summaries = {}
    for pc_id in pc_ind_summary:
        pc = pd.concat(pc_ind_summary[pc_id], names = ["Cell", "Resistor"]).reset_index()

        pc['Resistor'] = pc['Resistor'].map(res_dict)
        pc['Stage'] = pc['Cell'].map(stages)
        pc['Array'] = pc['Cell'].map(arrays)
        pc = calc2ndDeriv(pc, 'Voltage', 'Current')

        #display(pc_set, timings[pc_set][cells[0]])
        pc_summaries[pc_id] = pc
    return pc_summaries

def td_format(td_object):
    seconds = int(td_object.total_seconds())
    periods = [
        ('year',        60*60*24*365),
        ('month',       60*60*24*30),
        ('day',         60*60*24),
        ('hour',        60*60),
        ('minute',      60),
        #('second',      1)
    ]

    strings=[]
    for period_name, period_seconds in periods:
        if seconds > period_seconds:
            period_value , seconds = divmod(seconds, period_seconds)
            has_s = 's' if period_value > 1 else ''
            strings.append("%s %s%s" % (period_value, period_name, has_s))

    return ", ".join(strings)

def calc2ndDeriv(df, y_col, x_col):
    df['abs_y2'] = (
        (
            (
                (
                    df[y_col] - df[y_col].shift(1)
                ) / (
                    df[x_col] - df[x_col].shift(1)
                )
            ) - (
                (
                    df[y_col] - df[y_col].shift(1)
                ) / (
                    df[x_col] - df[x_col].shift(1)
                )
            ).shift(1)
        ) / (
            (
                0.5 * (
                    df[x_col] + df[x_col].shift(1)
                )
            ) - (
                0.5 * (
                    df[x_col] + df[x_col].shift(1)
                )
            ).shift(1)
        )
    ).abs()
    return df

## Import data

In [215]:
config.io_dir = Path("C:/Users/nms198/OneDrive - Newcastle University/3_Coding/Python/Ti_Rod_BES_CBL")
config.update = True

In [216]:
getConfigData()

Importing processed data...
Importing config...


## Set parameters

In [217]:
ChivasB = True

cells = [
    'A1', 'A2', 'A3', 'A4',
    'B1', 'B2', 'B3', 'B4',
    'C1', 'C2', 'C3', 'C4']

colours = [
    getColour('mid', 'red'),
    getColour('mid', 'green'),
    getColour('mid', 'blue'),
    getColour('mid', 'purple'),
]

shapes = [
    'circle',
    'square',
    'diamond',
]

## Setup

In [218]:
res_dict = makeResDict(ChivasB)

In [219]:
# Data columns
vols = {}
ocps = {}
rexts = {}
ocp_ds = {}
rext_ds = {}
stages = {}
arrays = {}

for cell in cells:
    vols[cell] = 'BES_' + cell + '__V'
    ocps[cell] = 'OCP_' + cell
    rexts[cell] = 'REXT_' + cell

    ocp_ds[cell] = 'OCP_' + cell + '__d'
    rext_ds[cell] = 'REXT_' + cell + '__d'
    stages[cell] = int(re.search(r'\d+', cell)[0])
    arrays[cell] = re.search('[a-zA-Z]+', cell)[0]

pd.DataFrame([vols, ocps, rexts, stages, arrays]).T

,0,1,2,3,4
A1,BES_A1__V,OCP_A1,REXT_A1,1,A
A2,BES_A2__V,OCP_A2,REXT_A2,2,A
A3,BES_A3__V,OCP_A3,REXT_A3,3,A
A4,BES_A4__V,OCP_A4,REXT_A4,4,A
B1,BES_B1__V,OCP_B1,REXT_B1,1,B
B2,BES_B2__V,OCP_B2,REXT_B2,2,B
B3,BES_B3__V,OCP_B3,REXT_B3,3,B
B4,BES_B4__V,OCP_B4,REXT_B4,4,B
C1,BES_C1__V,OCP_C1,REXT_C1,1,C
C2,BES_C2__V,OCP_C2,REXT_C2,2,C


## Create data subset with relevant columns

In [220]:
cols_list = ['DateTime'] + list(vols.values()) + list(ocps.values()) + list(rexts.values())
data = subsetDataCols(config.data['all_data'], cols_list)
data

,DateTime,BES_A1__V,BES_A2__V,BES_A3__V,BES_A4__V,BES_B1__V,BES_B2__V,BES_B3__V,BES_B4__V,BES_C1__V,...,REXT_A3,REXT_A4,REXT_B1,REXT_B2,REXT_B3,REXT_B4,REXT_C1,REXT_C2,REXT_C3,REXT_C4
0,2022-03-02 21:36:54+00:00,4.7,3.1,3.1,3.4,4.4,3.4,3.1,3.1,4.7,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
1,2022-03-02 21:37:53+00:00,4.7,3.4,3.4,3.1,4.4,3.4,3.7,3.4,4.4,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2,2022-03-02 21:38:53+00:00,4.4,3.7,3.1,3.4,4.4,3.7,3.7,3.4,4.4,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
3,2022-03-02 21:39:55+00:00,4.7,3.4,3.4,3.1,4.4,3.7,3.4,3.4,4.4,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
4,2022-03-02 21:40:55+00:00,5.7,3.4,3.1,3.4,4.1,3.7,3.4,3.4,4.4,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42320,2022-04-01 08:55:59+00:00,15.1,10.2,5.3,3.7,14.7,7.6,4.7,4.7,22.2,...,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0
42321,2022-04-01 08:56:58+00:00,15.4,9.9,5.3,4.1,15.1,7.9,5.3,5.0,21.9,...,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0
42322,2022-04-01 08:58:00+00:00,15.4,10.2,5.7,3.7,14.7,7.6,4.7,5.0,22.5,...,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0
42323,2022-04-01 08:59:00+00:00,15.4,9.9,5.3,3.4,15.1,7.9,5.0,5.0,21.9,...,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0


## Identify OCP/Rext changes

In [221]:
# Set 1 for OCP on, 0 for OCP off
# Set 1 for Res Increment, -1 for Res Decrement
for cell in cells:
    data[ocp_ds[cell]] = data[ocps[cell]] - data[ocps[cell]].shift(1)
    data[rext_ds[cell]] = data[rexts[cell]] - data[rexts[cell]].shift(1)
    data.loc[data[rext_ds[cell]] > 0, rext_ds[cell]] = 1
    data.loc[data[rext_ds[cell]] < 0, rext_ds[cell]] = -1


## Identify Timings of OCP on/off, followed by Rext increment

In [222]:
same_pc_threshold = timedelta(minutes=30)

timings = getTimings(data)
aligned_timings = alignTimings(timings)

aligned_timings[cells[0]]

,OCP_ON,OCP_ON_index,OCP_OFF,OCP_OFF_index,RES_INC,RES_INC_index,DURATION
0,2022-03-15 07:41:47,17872,2022-03-15 09:36:53,17987,2022-03-15 12:01:57,18132,0 days 04:20:10
1,2022-03-16 07:30:01,19283,2022-03-16 09:30:05,19403,2022-03-16 12:19:12,19572,0 days 04:49:11
2,2022-03-16 13:46:02,19654,2022-03-16 13:47:03,19655,2022-03-16 14:02:46,19669,0 days 00:16:44
3,2022-03-17 00:45:02,20311,2022-03-17 02:55:49,20440,2022-03-17 05:45:56,20610,0 days 05:00:54
4,2022-03-18 00:45:35,21743,2022-03-18 02:55:38,21873,2022-03-18 05:45:44,22043,0 days 05:00:09
5,2022-03-18 16:48:55,22698,2022-03-18 16:57:54,22707,2022-03-18 17:42:46,22749,0 days 00:53:51
6,2022-03-19 00:42:00,23168,2022-03-19 02:52:02,23298,2022-03-19 05:42:27,23464,0 days 05:00:27
7,2022-03-19 16:48:02,24130,2022-03-19 16:57:46,24138,2022-03-19 17:42:03,24183,0 days 00:54:01
8,2022-03-20 00:42:57,24602,2022-03-20 02:52:02,24731,2022-03-20 05:42:46,24895,0 days 04:59:49
9,2022-03-20 16:48:14,25560,2022-03-20 16:57:15,25569,2022-03-20 17:42:45,25611,0 days 00:54:31


## Get individual PC data and create Voltage/Current/Power/RInt summary

In [223]:
pc_sums = createPCSums()

Dropping PC_ID = 2


In [224]:
y2_thresh = 0.1
date = {}
dur = {}
max_C = {}
max_P = {}
P_res = {}
R_int = {}
maxes = {}

#For each polarisation curve
for pc_id in pc_sums:
    date[pc_id] = {}
    dur[pc_id] = {}
    max_C[pc_id] = {}
    max_P[pc_id] = {}
    P_res[pc_id] = {}
    R_int[pc_id] = {}
    for cell in cells:
        #Set Date
        date[pc_id][cell] = aligned_timings[cell].query('index == @pc_id')['OCP_ON'][pc_id]
        #Set duration
        dur[pc_id][cell] = aligned_timings[cell].query('index == @pc_id')['RES_INC'][pc_id] - aligned_timings[cell].query('index == @pc_id')['OCP_ON'][pc_id]
        #Find max current
        max_C[pc_id][cell] = max(pc_sums[pc_id].loc[pc_sums[pc_id]['Cell'] == cell].Current)
        #Find max power
        max_P[pc_id][cell] = max(pc_sums[pc_id].loc[pc_sums[pc_id]['Cell'] == cell].Power)
        #Assign resistor associated with max power
        try:
            P_res[pc_id][cell] = pc_sums[pc_id].loc[pc_sums[pc_id]['Cell'] == cell].loc[pc_sums[pc_id].loc[pc_sums[pc_id]['Cell'] == cell].Power == max_P[pc_id][cell]].Resistor.values[0]
        except:
            P_res[pc_id][cell] = np.nan
        #Subset data for RInt calculation
        R_int_temp = deepcopy(pc_sums[pc_id].loc[pc_sums[pc_id]['Cell'] == cell])
        R_int_temp['belowT'] = R_int_temp['abs_y2'] < y2_thresh
        start = R_int_temp.belowT.eq(True).idxmax()
        R_int_temp = R_int_temp.truncate(before=start)#, after=4)
        end = R_int_temp.belowT.eq(False).idxmax()
        R_int_temp = R_int_temp.truncate(after=end-1)
        #Calculate R_Int
        if len(R_int_temp) > 1:
            R_int[pc_id][cell] = -stats.linregress(R_int_temp.Current, R_int_temp.Voltage).slope * 1000
        else:
            R_int[pc_id][cell] = np.nan
    #Assign to table
    maxes[pc_id] = pd.DataFrame.from_dict({"DateTime":date[pc_id], "Duration":dur[pc_id], "Max Current":max_C[pc_id], "Max Power":max_P[pc_id], "Max P Resistor": P_res[pc_id], "RInt Estimate": R_int[pc_id]}, orient='columns')
    display(maxes[pc_id])

,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-15 07:41:47,0 days 04:20:10,245.871560,23.960169,1004.0,1469.658049
A2,2022-03-15 07:41:47,0 days 04:20:10,221.052632,23.254821,1004.0,1809.827576
A3,2022-03-15 07:41:47,0 days 04:20:10,172.266140,23.652201,1004.0,2082.063936
A4,2022-03-15 07:41:47,0 days 04:20:10,133.333333,20.120080,2004.0,2642.216714
B1,2022-03-15 07:41:47,0 days 04:20:10,188.925081,17.275787,1004.0,2017.543823
B2,2022-03-15 07:41:47,0 days 04:20:10,165.019763,19.466175,1004.0,2262.313127
B3,2022-03-15 07:42:47,0 days 04:19:10,175.559947,22.171952,1004.0,2112.136243
B4,2022-03-15 07:42:47,0 days 04:19:10,121.513944,18.280419,2004.0,2831.038081
C1,2022-03-15 07:42:47,0 days 04:19:10,229.629630,20.196972,1004.0,2148.727420
C2,2022-03-15 07:42:47,0 days 04:19:10,231.578947,21.318416,1004.0,1873.480418


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-16 07:30:01,0 days 04:49:11,229.629630,15.388934,1004.0,1302.302612
A2,2022-03-16 07:30:01,0 days 04:49:11,176.146789,13.149412,1004.0,4895.583355
A3,2022-03-16 07:30:01,0 days 04:49:11,163.302752,15.264476,2004.0,4451.988361
A4,2022-03-16 07:30:01,0 days 04:49:11,111.111111,11.635482,5100.0,4134.453782
B1,2022-03-16 07:30:01,0 days 04:49:11,177.777778,9.934736,2004.0,4323.773585
B2,2022-03-16 07:30:01,0 days 04:49:11,133.944954,11.167745,2004.0,4385.188867
B3,2022-03-16 07:30:01,0 days 04:49:11,155.555556,13.079646,2004.0,4827.480916
B4,2022-03-16 07:30:01,0 days 04:49:11,84.210526,10.663184,5100.0,4093.387178
C1,2022-03-16 07:30:01,0 days 04:49:11,229.629630,14.421158,2004.0,2270.029674
C2,2022-03-16 07:30:01,0 days 04:49:11,200.000000,12.785747,1004.0,4074.679943


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-17 00:45:02,0 days 05:00:54,274.074074,17.689329,514.5,1198.854726
A2,2022-03-17 00:45:02,0 days 05:00:54,205.863192,19.382719,1004.0,6028.297342
A3,2022-03-17 00:45:02,0 days 05:00:54,185.964912,19.464197,2004.0,5363.784514
A4,2022-03-17 00:45:02,0 days 05:00:54,135.082604,18.395210,2004.0,2781.488718
B1,2022-03-17 00:45:02,0 days 05:00:54,207.407407,16.039452,1004.0,5328.721374
B2,2022-03-17 00:45:02,0 days 05:00:54,163.461538,16.930958,2004.0,5926.330784
B3,2022-03-17 00:45:02,0 days 05:00:54,207.407407,17.170783,2004.0,5721.383318
B4,2022-03-17 00:45:02,0 days 05:00:54,121.282799,16.060060,2004.0,3079.908867
C1,2022-03-17 00:45:02,0 days 05:00:54,221.052632,16.191484,1004.0,2564.064489
C2,2022-03-17 00:45:02,0 days 05:00:54,234.862385,18.233157,1004.0,4724.161533


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-18 00:45:35,0 days 05:00:09,303.703704,25.466145,1004.0,1020.497435
A2,2022-03-18 00:45:35,0 days 05:00:09,251.851852,21.435149,1004.0,5282.957971
A3,2022-03-18 00:45:35,0 days 05:00:09,207.407407,20.827590,2004.0,4984.634829
A4,2022-03-18 00:45:35,0 days 05:00:09,143.245870,19.405110,2004.0,2624.500894
B1,2022-03-18 00:45:35,0 days 05:00:09,251.851852,19.661604,1004.0,5192.229704
B2,2022-03-18 00:45:35,0 days 05:00:09,177.777778,18.395210,2004.0,5198.728754
B3,2022-03-18 00:45:35,0 days 05:00:09,200.000000,18.587325,2004.0,5245.003231
B4,2022-03-18 00:45:35,0 days 05:00:09,124.392614,16.711078,2004.0,3015.221917
C1,2022-03-18 00:45:35,0 days 05:00:09,274.074074,17.618526,1004.0,2306.752412
C2,2022-03-18 00:45:35,0 days 05:00:09,277.064220,20.941235,1004.0,4138.591292


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-18 16:48:55,0 days 00:53:51,366.123779,27.847682,755.0,157.933488
A2,2022-03-18 16:48:55,0 days 00:53:51,211.009174,16.293658,2004.0,1261.513067
A3,2022-03-18 16:48:55,0 days 00:53:51,185.016287,17.059885,2004.0,1108.826648
A4,2022-03-18 16:48:55,0 days 00:53:51,144.625407,15.881517,2004.0,1195.225313
B1,2022-03-18 16:48:55,0 days 00:53:51,251.851852,17.115289,2004.0,1156.610960
B2,2022-03-18 16:48:55,0 days 00:53:51,177.777778,15.369386,2004.0,1354.324508
B3,2022-03-18 16:48:55,0 days 00:53:51,181.651376,15.474656,2004.0,1421.370004
B4,2022-03-18 16:48:55,0 days 00:53:51,133.944954,13.983413,2004.0,1131.222333
C1,2022-03-18 16:48:55,0 days 00:53:51,274.074074,20.563373,2004.0,NaN
C2,2022-03-18 16:48:55,0 days 00:53:51,274.074074,18.089900,2004.0,NaN


...

,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-23 00:54:01,0 days 04:48:47,282.568807,26.723546,1004.0,1324.239034
A2,2022-03-23 00:54:01,0 days 04:48:47,216.513761,21.435149,1004.0,5052.943130
A3,2022-03-23 00:54:01,0 days 04:48:47,155.555556,19.800719,2004.0,5140.284689
A4,2022-03-23 00:54:01,0 days 04:48:47,128.440367,17.730664,2004.0,4937.311300
B1,2022-03-23 00:54:01,0 days 04:48:47,181.651376,18.032939,2004.0,2406.714777
B2,2022-03-23 00:54:01,0 days 04:48:47,155.555556,18.032939,2004.0,5004.255638
B3,2022-03-23 00:54:01,0 days 04:48:47,155.555556,17.412295,2004.0,5235.397867
B4,2022-03-23 00:54:01,0 days 04:48:47,114.868805,16.638104,2004.0,3220.557776
C1,2022-03-23 00:54:01,0 days 04:48:47,251.851852,23.066367,2004.0,NaN
C2,2022-03-23 00:54:01,0 days 04:48:47,348.623853,24.644711,1004.0,3648.540227


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-23 17:03:03,0 days 00:39:43,325.925926,22.838795,514.5,NaN
A2,2022-03-23 17:03:03,0 days 00:39:43,229.629630,16.765913,2004.0,1004.685520
A3,2022-03-23 17:03:03,0 days 00:39:43,177.777778,15.934975,2004.0,973.044933
A4,2022-03-23 17:03:03,0 days 00:39:43,138.110749,13.717385,2004.0,1069.640319
B1,2022-03-23 17:03:03,0 days 00:39:43,188.925081,15.774870,2004.0,1125.316345
B2,2022-03-23 17:03:03,0 days 00:39:43,165.472313,14.523134,2004.0,1017.099506
B3,2022-03-23 17:03:03,0 days 00:39:43,157.798165,13.503119,2004.0,1063.649364
B4,2022-03-23 17:03:03,0 days 00:39:43,138.110749,13.717385,2004.0,1040.374598
C1,2022-03-23 17:03:03,0 days 00:39:43,282.568807,23.346153,2004.0,1029.060926
C2,2022-03-23 17:03:03,0 days 00:39:43,370.370370,20.460175,514.5,NaN


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-25 01:55:58,0 days 03:39:47,294.462541,34.643675,1004.0,1260.255621
A2,2022-03-25 01:55:58,0 days 03:39:47,237.483531,30.468137,1004.0,1667.923970
A3,2022-03-25 01:55:58,0 days 03:39:47,216.073781,30.468137,1004.0,1816.239794
A4,2022-03-25 01:55:58,0 days 03:39:47,197.473275,30.001517,2004.0,1927.285551
B1,2022-03-25 01:55:58,0 days 03:39:47,216.286645,25.275388,1004.0,1824.672422
B2,2022-03-25 01:55:58,0 days 03:39:47,207.407407,27.380120,1004.0,1891.836772
B3,2022-03-25 01:55:58,0 days 03:39:47,217.391304,26.854223,1004.0,1806.111797
B4,2022-03-25 01:55:58,0 days 03:39:47,196.890185,30.468268,2004.0,1926.260453
C1,2022-03-25 01:55:58,0 days 03:39:47,288.073394,30.468137,1004.0,1565.782084
C2,2022-03-25 01:55:58,0 days 03:39:47,317.431193,27.380120,1004.0,1394.170418


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-28 02:55:32,0 days 03:40:10,383.486239,42.266932,1004.0,1102.596443
A2,2022-03-28 02:55:32,0 days 03:40:10,324.770642,35.240647,1004.0,1349.013205
A3,2022-03-28 02:55:32,0 days 03:40:10,273.615635,33.794462,1004.0,1564.889809
A4,2022-03-28 02:55:32,0 days 03:40:10,255.599473,35.729442,1004.0,1586.676713
B1,2022-03-28 02:55:32,0 days 03:40:10,311.926606,32.163436,1004.0,1407.779441
B2,2022-03-28 02:55:32,0 days 03:40:10,273.615635,33.465030,1004.0,1576.295789
B3,2022-03-28 02:55:32,0 days 03:40:10,264.220183,29.124502,1004.0,2415.221987
B4,2022-03-28 02:55:32,0 days 03:40:10,263.175231,37.718287,1004.0,1533.317842
C1,2022-03-28 02:55:32,0 days 03:40:10,365.137615,32.738735,1004.0,1340.015705
C2,2022-03-28 02:55:32,0 days 03:40:10,370.370370,30.224741,1004.0,NaN


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-31 02:55:14,0 days 03:40:15,128.787879,17.543039,2004.0,3250.572921
A2,2022-03-31 02:55:14,0 days 03:40:15,91.739553,9.759531,5100.0,2689.603214
A3,2022-03-31 02:55:14,0 days 03:40:15,82.798834,9.233137,5100.0,NaN
A4,2022-03-31 02:55:14,0 days 03:40:15,84.742468,9.344096,5100.0,NaN
B1,2022-03-31 02:55:14,0 days 03:40:15,97.959184,12.840159,5100.0,3844.956806
B2,2022-03-31 02:55:14,0 days 03:40:15,79.689018,9.284267,5100.0,NaN
B3,2022-03-31 02:55:14,0 days 03:40:15,81.481481,7.702596,5100.0,4447.958601
B4,2022-03-31 02:55:14,0 days 03:40:15,86.686103,10.192941,5100.0,NaN
C1,2022-03-31 02:55:14,0 days 03:40:15,134.201954,16.263529,5100.0,3466.851166
C2,2022-03-31 02:55:14,0 days 03:40:15,121.100917,14.968959,5100.0,2124.114952


## Create Plots

In [225]:
def markerLinesTrace(x_par, y_par, shape_par, color_par, data, stage, array):
    trace1 = trace_base
    trace1.update(
        x=data[x_par], y=data[y_par],
        marker_symbol = shapes[array],
        mode = "markers+lines",
        line=dict(
            color=colours[stage],
            width=2,
            dash="dash",
            #shape=par_info['line'][0]
        ),
        #connectgaps=False,
        showlegend=True)
    return trace1

def markerLines2ndTrace(x_par, y_par, shape_par, color_par, data, stage, array):
    trace1 = trace_base
    trace1.update(
        x=data[x_par], y=data[y_par],
        marker_symbol = shapes[array] + "-open",
        mode = "markers+lines",
        line=dict(
            color=colours[stage],
            width=2,
            dash="solid",
            #shape=par_info['line'][0]
        ),
        #connectgaps=False,
        showlegend=False)
    return trace1

def trendTrace(x_par, y_par, data, c):
    x_pred, y_pred, trend = fitLR(x_par, y_par, data)
    hover_header = makeTrendHover(trend)

    trace2 = trace_base
    trace2.update(
        x=x_pred, y=y_pred,
        mode = "lines",
        line=dict(
            color=config.config['info']['colours']['rgba_str'][c],
            width=2,
            #dash=par_info['dash'][0],
            #shape=par_info['line'][0]
        ),
        connectgaps=True,
        showlegend=False,
        hovertemplate=hover_header)
    return trace2

In [226]:
figs = {}
for pc_id in pc_sums:
    cols = ['Voltage', 'Power']
    
    figs[pc_id] = make_subplots(rows=1, cols=3, specs=[[{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}]])
    col= 1
    for array in sorted(list(set(arrays.values()))):
        a = 0
        for stage in sorted(list(set(stages.values()))):
            trace_base = go.Scatter(x=[], y=[],
                            name=array + str(stage), 
                            legendgroup=array)

            data = pc_sums[pc_id].loc[pc_sums[pc_id]['Array'] == array].loc[pc_sums[pc_id]['Stage'] == stage]
            x_par = 'Current'
            y_par = 'Voltage'
            shape_par = "Stage"
            color_par = "Stage"
            figs[pc_id].add_trace(markerLinesTrace(x_par, y_par, shape_par, color_par, data, a, col-1), row=1, col=col)
            #fig.add_trace(markerLinesTrace(x_par, "abs_y2", shape_par, color_par, data, a, col-1), row=1, col=col)
            y_par = 'Power'
            figs[pc_id].add_trace(markerLines2ndTrace(x_par, y_par, shape_par, color_par, data, a, col-1), row=1, col=col, secondary_y=True)
            a += 1
        col += 1
    figs[pc_id].update_layout(
            title=maxes[pc_id].loc[cells[0], 'DateTime'].strftime("%d/%m/%Y %H:%M") + " (" + td_format(maxes[pc_id].loc[cells[0], 'Duration']) + ")",
            margin=dict(l=5, r=5, b=5, t=30, pad=0),
            template="simple_white",
            #paper_bgcolor='rgba(0,0,0,0)',
            font=dict(
                family="Arial",
                color="black"
            ),
            width=1300,
            height=330,
        )
    figs[pc_id].update_yaxes(title_text="Voltage (mV)", mirror=True, secondary_y=False, matches='y')
    figs[pc_id].update_yaxes(title_text="Power (uW)", mirror=True, secondary_y=True, matches='y2')
    figs[pc_id].update_xaxes(title_text="Current (uA)", mirror=True, matches='x')
    figs[pc_id].show()

## Latest Data

In [232]:
pc_id = len(maxes)

display(pc_sums[pc_id])
display(maxes[pc_id])

figs[pc_id].update_yaxes(range = [0,500], secondary_y=False, matches='y') # Voltage
figs[pc_id].update_yaxes(range = [0,50], secondary_y=True, matches='y2') # Power
figs[pc_id].update_xaxes(range = [0,400], matches='x') #Current
figs[pc_id]

,Cell,Resistor,Voltage,Current,Power,Stage,Array,abs_y2
0,A1,NaN,463.6,0.000000,0.000000,1,A,NaN
1,A1,51000.0,444.8,8.721569,3.879354,1,A,NaN
2,A1,15000.0,389.4,25.960000,10.108824,1,A,0.081523
3,A1,5100.0,293.1,57.470588,16.844629,1,A,0.006467
4,A1,2004.0,187.5,93.562874,17.543039,1,A,0.003854
...,...,...,...,...,...,...,...,...
127,C4,514.5,39.7,77.162293,3.063343,4,C,0.535671
128,C4,303.6,21.9,72.134387,1.579743,4,C,19.799916
129,C4,153.5,10.5,68.403909,0.718241,4,C,0.110599
130,C4,54.5,3.4,62.385321,0.212110,4,C,0.384904


,DateTime,Duration,Max Current,Max Power,Max P Resistor,RInt Estimate
A1,2022-03-31 02:55:14,0 days 03:40:15,128.787879,17.543039,2004.0,3250.572921
A2,2022-03-31 02:55:14,0 days 03:40:15,91.739553,9.759531,5100.0,2689.603214
A3,2022-03-31 02:55:14,0 days 03:40:15,82.798834,9.233137,5100.0,NaN
A4,2022-03-31 02:55:14,0 days 03:40:15,84.742468,9.344096,5100.0,NaN
B1,2022-03-31 02:55:14,0 days 03:40:15,97.959184,12.840159,5100.0,3844.956806
B2,2022-03-31 02:55:14,0 days 03:40:15,79.689018,9.284267,5100.0,NaN
B3,2022-03-31 02:55:14,0 days 03:40:15,81.481481,7.702596,5100.0,4447.958601
B4,2022-03-31 02:55:14,0 days 03:40:15,86.686103,10.192941,5100.0,NaN
C1,2022-03-31 02:55:14,0 days 03:40:15,134.201954,16.263529,5100.0,3466.851166
C2,2022-03-31 02:55:14,0 days 03:40:15,121.100917,14.968959,5100.0,2124.114952
